In [3]:
# Load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('ex4data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
X = data['X']
Y = data['y']

# Training data of 5000 20x20 pixel images
X.shape

(5000, 400)

In [5]:
# We should one-hot encode our label
# We can use a utility function from scikit
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(Y)

y_onehot[0], Y[0]

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([10], dtype=uint8))

## Our Neural Network
* 400 input units (+ the bias unit)
* hidden layer with 25 nodes (+1 bias unit)
* output layer with 10 units

In [ ]:
# Functions required to compute the cost
def sigmoid(z):
    return 1 / ((1 + np.exp(-z)))

def forward_propagate(X, theta1, theta2):
    m = X.shape[0]  # number of training samples
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)  # insert our bias unit column
    
    
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)  # compute activation in input layer and add bias column
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [26]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]  # number of training examples
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape parameter array into parameters for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost for one iteration
    # note that this is unregularized
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
        
    J = J / m
    
    # Regularizing our parameters
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))  
    return J

In [27]:
# test our implementation of computing cost 
input_size = 400  # 20x20 pixel values
hidden_size = 25
num_labels = 10
learning_rate = 1

# randomly initialize parameters for the whole network
params = (np.random.random(size = ((input_size + 1) * hidden_size) + ((hidden_size + 1) * num_labels)) - 0.5) * 0.25

cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)


7.1568691648586427